In [ ]:
print("Hola Mundo desde Jupyter")

In [ ]:
pip install psycopg2

In [ ]:
pip install plotly

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


# Conexion local con bd Postgres y Ploty para Graficar

In [ ]:
import psycopg2
import plotly.graph_objs as  go
#Establecer la conexion a la base de datos de Postgres de manera local
conn = psycopg2.connect(
    database="bdcarsales",
    user ="admin",
    password="admin",
    host="localhost",
    port="5432"
)
#Crear un cursor para ejecutar consultas SQL
cur = conn.cursor()
#Ejecutar una consulta  SQL para obtener los datos que desear graficar
cur.execute("SELECT nombre,stock from autos")
#Extraer los datos de la consulta
filas = cur.fetchall()
#Cerrar el cursor y la conexion a la BD
cur.close()
conn.close()

#Separando los datos en dos listas (nombres y stock)
nombres = [fila[0] for fila in filas]
stock = [fila[1] for fila in filas]

#Crear un grafico de barras utilizando plotly
data = [go.Bar(x=nombres,y=stock)]

layout = go.Layout(
    title="Stock Actual de Automoviles",
    xaxis=dict(title="Nombres"),
    yaxis=dict(title="Stock")
    
)

fig = go.Figure(data=data,layout=layout)

#Pintar el Graficos

fig.show()


# Agrupar a los automoviles por categoria

In [20]:
import psycopg2
import plotly.express as px
import pandas as pd

#conectar a la base de datos
try:
    connection = psycopg2.connect(
        database="bdcarsales",
        user="admin",
        password="admin",
        host="localhost",
        port="5432"
    )
    
    cursor=connection.cursor()
    consulta_sql="SELECT categoria , COUNT(*) as cantidad from autos GROUP BY categoria";
    cursor.execute(consulta_sql)
    resultados=cursor.fetchall()
    
    #Crear un dataFrame para los resultados    
    df = pd.DataFrame(resultados,columns=['categoria','cantidad'])
    #Crear el Grafico de Barras con Plotly
    fig=px.bar(df,x='categoria',y='cantidad',color="categoria",title='Cantidad de Autos por Categoria')
    #Mostrar el Grafico
    fig.show()
    
except (Exception ,psycopg2.Error)as error:
    print("error al conectarse a la BD ",error)
finally:
    #Cerrar ka conexion a la base de datos
    if connection:
        cursor.close()
        connection.close()    

In [4]:
#Ejmplos de Pandas
import pandas as pd
data = [
    {"Nombre":"Juan","Edad":25},
    {"Nombre":"Maria","Edad":30},
    {"Nombre":"Luis","Edad":22},
]

df=pd.DataFrame(data)

mayores_de_25 = df[df["Edad"]>25]
mayores_de_25


,Nombre,Edad
1,Maria,30


# Agrupar a los automoviles por categoria PIE

In [25]:
import psycopg2
import plotly.express as px
import pandas
#conectarse a la base de datos
conn = psycopg2.connect(
        database="bdcarsales",
        user="admin",
        password="admin",
        host="localhost",
        port="5432"
    )
sql_query="SELECT categoria , COUNT(*) as cantidad from autos GROUP BY categoria"
df= pd.read_sql_query(sql_query,conn)
conn.close()

#Crear un Grafico de pastel con plotly express
fig=px.pie(df,names='categoria',values='cantidad',title='Grafico de Pastel de Automoviles agrupados por categoria')
fig.show()



C:\Users\Armando\AppData\Local\Temp\ipykernel_9660\1531521354.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# Conectarse a un Servicio y mostrar resultados en Plotly

In [30]:
import requests
import plotly.express as px
#URL de la API que vamos a consumir
api_url="http://127.0.0.1:5000/autos"
#realizamos una solicitud GET a la API
response=requests.get(api_url)
#Verificamos si la solicitud fue exitosa (CODIGO DE ESTADO 200)
if response.status_code==200:
    #convertimos los datos a JSON en un diccionario
    data=response.json()
    #Accedemos a los resultados dentro del objeto del results
    results=data.get("results",[])#si results no existe asignamos una lista vacia
    if results:
        df=pd.DataFrame(results)
        #Creamos el grafico de barras con plotly express
        fig=px.bar(df,x='nombre',y='puertas',title='Grafico de Barras desde API con Plotly')
        fig.show()
    else:
        print("No se encontraron resultados en la API")
else:
    print("Error al realizxar la solicitud a la API")